## CS-470: Homework Set # 2: Schedule for Naive Loop
#### Name: GAO Xufeng. Sciper: 337088

In [1]:
import json
import numpy as np
import copy
import ctypes

In [2]:
class schedule_cell:
    
    def __init__(self, addr=None):
        self.addr = addr 
        self.ALU0 = 'nop'
        self.ALU1 = 'nop'
        self.Mult = 'nop'
        self.Mem = 'nop'
        self.Branch = 'nop'
    
    def set_slot(self, slot_type, ID):
        if slot_type == 'ALU0':
            self.ALU0 = ID
        if slot_type == 'ALU1':
            self.ALU1 = ID
        if slot_type == 'Mult':
            self.Mult = ID
        if slot_type == 'Mem':
            self.Mem = ID
        if slot_type == 'Branch':
            self.Branch = ID
            
    def append_consume_reg_slot(self, i, slot_type, consume_reg):
        if slot_type == 'ALU0':
            self.ALU0[i].append(consume_reg)
        if slot_type == 'ALU1':
            self.ALU1[i].append(consume_reg)
        if slot_type == 'Mult':
            self.Mult[i].append(consume_reg)
        if slot_type == 'Mem':
            self.Mem[i].append(consume_reg)
            
    def insert_clean_dep_slot(self, slot_type, clean_dep):
        if slot_type == 'ALU0':
            self.ALU0 = clean_dep
        if slot_type == 'ALU1':
            self.ALU1 = clean_dep
        if slot_type == 'Mult':
            self.Mult = clean_dep
        if slot_type == 'Mem':
            self.Mem = clean_dep     
        
            
    def insert_imm(self, i, slot_type, imm_ind, imm):
        if slot_type == 'ALU0':
            self.ALU0[i].insert(imm_ind, imm)
        if slot_type == 'ALU1':
            self.ALU1[i].insert(imm_ind, imm)
        if slot_type == 'Mult':
            self.Mult[i].insert(imm_ind, imm)
        if slot_type == 'Mem':
            self.Mem[i][imm_ind] = str(imm)+'('+self.Mem[i][imm_ind]+')'
            
    def return_ID(self, slot_type):
        ID = 'nop'
        if slot_type == 'ALU0' and not self.ALU0_empty():
            ID = self.ALU0
        if slot_type == 'ALU1' and not self.ALU1_empty():
            ID = self.ALU1 
        if slot_type == 'Mult' and not self.Mult_empty():
            ID = self.Mult
        if slot_type == 'Mem' and not self.Mem_empty():
            ID = self.Mem 
        if slot_type == 'Branch' and not self.Branch_empty():
            ID = self.Branch
        return ID
        
    def set_addr(self, addr):
        self.addr = addr        
    def ALU0_empty(self):
        return self.ALU0 == 'nop'
    def ALU1_empty(self):
        return self.ALU1 == 'nop'
    def Mult_empty(self):
        return self.Mult == 'nop'
    def Mem_empty(self):
        return self.Mem == 'nop'
    def Branch_empty(self):
        return self.Branch == 'nop'
    
    def show_schedule_cell(self):
        print('addr: {}, ALU0: {}, ALU1: {}, Mult: {}, Mem: {}, Branch: {}'.format(self.addr, self.ALU0, self.ALU1, self.Mult, self.Mem, self.Branch))
    
    def output_schedule_cell(self, slot_type):
        cell = 'nop'
        if slot_type == 'ALU0' and not self.ALU0_empty():
            for _, value in self.ALU0.items():
                cell = value
                cell = (cell[0]) + ' ' + ', '.join(char for char in cell[1:None])
        if slot_type == 'ALU1' and not self.ALU1_empty():
            for _, value in self.ALU1.items():
                cell = value
                cell = (cell[0]) + ' ' + ', '.join(char for char in cell[1:None])
        if slot_type == 'Mult' and not self.Mult_empty():
            for _, value in self.Mult.items():
                cell = value
                cell = (cell[0]) + ' ' + ', '.join(char for char in cell[1:None])
        if slot_type == 'Mem' and not self.Mem_empty():
            for _, value in self.Mem.items():
                cell = value
                cell = (cell[0]) + ' ' + ', '.join(char for char in cell[1:None])
        if slot_type == 'Branch' and not self.Branch_empty():
            for _, value in self.Branch.items():
                cell = value
                cell = (cell[0]) + ' ' + ', '.join(char for char in cell[1:None])
    
        return cell
    
        

In [3]:
class Compiler_state:
    def __init__(self):
        
        # input instruction sets
        self.Ins_set = []
        self.loop_region = None
        self.BB0 = None
        self.BB1 = None
        self.BB2 = None
        self.ID = 0
        self.Dependency_tab = {'Instr_Addr':[], 'ID':[], 'opcode':[], 'Des_reg':[], 'Local_dep':[],\
                               'InterLoop_dep':[], 'LoopInvar_dep':[], 'PostLoop_dep':[],\
                               'Consume_reg':[]}
        self.schedule_tab = []
        self.schedule_addr_tab = {}
        self.BB0_end = 0
        self.BB1_start = 0
        self.BB1_end = 0
        self.BB2_start = 0
        self.register_allocation_tab = []
        self.register_rename_tab = {}
        self.phase_3_Ins = []
        self.output_bundles = []
        
    # generate code blocks
    def generate_block(self):
        self.BB0 = self.Ins_set[0:self.loop_region[0]]
        self.BB1 = self.Ins_set[self.loop_region[0]:self.loop_region[1]+1]
        self.BB2 = self.Ins_set[self.loop_region[1]+1:None]
    
    # show code blocks
    def show_block(self):
        for BB0_ins in self.BB0:
            print(BB0_ins['PC'], ': ', BB0_ins['Ins'])
        print('----------')
        for BB1_ins in self.BB1:
            print(BB1_ins['PC'], ': ', BB1_ins['Ins'])
        print('----------')
        for BB2_ins in self.BB2:
            print(BB2_ins['PC'], ': ', BB2_ins['Ins'])
        
    # find Des_reg
    def find_Des_reg(self, ins):
        opcode: str = ins[0]
        assert opcode in ['add', 'addi', 'sub', 'mulu', 'ld', 'st', 'loop',\
                          'nop', 'mov'], "Undefined instruction: {}".format(i)
        if opcode in ['add', 'addi', 'sub', 'mulu', 'ld', 'mov']:
            return ins[1]
        else:
            return None
        
    # find consumed operand in each instruction
    def find_Consume_reg(self, ins):
        opcode: str = ins[0]
        assert opcode in ['add', 'addi', 'sub', 'mulu', 'ld', 'st', 'loop',\
                          'nop', 'mov'], "Undefined instruction: {}".format(i)
        Consume_reg = []
        if opcode in ['add', 'addi', 'sub', 'mulu']:
            Consume_reg.append(ins[2])
            if opcode != 'addi':
                Consume_reg.append(ins[3])
        elif opcode in ['ld', 'st']:
            if opcode == 'st':
                Consume_reg.append(ins[1])
            Consume_reg.append(ins[2].split('(')[1].strip(')'))
        elif opcode == 'mov':
            if ins[1].startswith('x') and ins[2].startswith('x'):
                Consume_reg.append(ins[2])

        return Consume_reg
    
    # find Local Dependencies
    def find_Local_dep(self):
        BB0_ind = (0, self.loop_region[0])
        BB1_ind = (self.loop_region[0], self.loop_region[1]+1)
        BB2_ind = (self.loop_region[1]+1, self.Dependency_tab['Instr_Addr'][-1]+1)
        for (start, end) in [BB0_ind, BB1_ind, BB2_ind]:
            for i in range(start, end):
                Consume_reg = self.Dependency_tab['Consume_reg'][i]
                if bool(Consume_reg):
                    Produce_reg = self.Dependency_tab['Des_reg'][start:i]
                    Produce_ID = self.Dependency_tab['ID'][start:i]
                    Produce_Instr_Addr = self.Dependency_tab['Instr_Addr'][start:i]
                    for reg in Consume_reg:
                        if reg in Produce_reg:
                            deps = [i for i, x in enumerate(Produce_reg) if x == reg][-1]
                            self.Dependency_tab['Local_dep'][i][reg] = Produce_ID[deps]
     
    # find InterLoop Dependencies
    def find_InterLoop_dep(self):
        BB0_ind = (0, self.loop_region[0])
        BB1_ind = (self.loop_region[0], self.loop_region[1]+1)
        
        Produce_reg_BB0 = self.Dependency_tab['Des_reg'][BB0_ind[0]:BB0_ind[1]]
        Produce_ID_BB0 = self.Dependency_tab['ID'][BB0_ind[0]:BB0_ind[1]]
        Produce_Instr_Addr_BB0 = self.Dependency_tab['Instr_Addr'][BB0_ind[0]:BB0_ind[1]]
    
        for i in range(BB1_ind[0], BB1_ind[1]):
            Consume_reg = self.Dependency_tab['Consume_reg'][i]
            if bool(Consume_reg):
                Producer_reg_before = self.Dependency_tab['Des_reg'][BB1_ind[0]:i]
                Produce_reg_after = self.Dependency_tab['Des_reg'][i:BB1_ind[1]]
                Produce_ID_after = self.Dependency_tab['ID'][i:BB1_ind[1]]
                Produce_Instr_Addr_after = self.Dependency_tab['Instr_Addr'][i:None]
                
                for reg in Consume_reg:
                    if reg not in Producer_reg_before and reg in Produce_reg_after:
                        deps = [i for i, x in enumerate(Produce_reg_after) if x == reg][-1]
                        self.Dependency_tab['InterLoop_dep'][i][reg] = {'BB1':Produce_ID_after[deps]}
                        if reg in Produce_reg_BB0:
                            deps = [i for i, x in enumerate(Produce_reg_BB0) if x == reg][-1]
                            self.Dependency_tab['InterLoop_dep'][i][reg]['BB0'] = Produce_ID_BB0[deps]
                            
    
    # find Loop Invariant Dependencies
    def find_LoopInvar_dep(self):
        BB0_ind = (0, self.loop_region[0])
        BB1_ind = (self.loop_region[0], self.loop_region[1]+1)
        BB2_ind = (self.loop_region[1]+1, self.Dependency_tab['Instr_Addr'][-1]+1)
        
        Produce_reg_BB0 = self.Dependency_tab['Des_reg'][BB0_ind[0]:BB0_ind[1]]
        Produce_ID_BB0 = self.Dependency_tab['ID'][BB0_ind[0]:BB0_ind[1]]
        Produce_Instr_Addr_BB0 = self.Dependency_tab['Instr_Addr'][BB0_ind[0]:BB0_ind[1]]
        Produce_reg_BB1 = self.Dependency_tab['Des_reg'][BB1_ind[0]:BB1_ind[1]]
        Produce_reg_BB2 = self.Dependency_tab['Des_reg'][BB2_ind[0]:BB2_ind[1]]
            
        for i in range(BB1_ind[0], BB1_ind[1]):
            Consume_reg = self.Dependency_tab['Consume_reg'][i]
            if bool(Consume_reg):
                for reg in Consume_reg:
                    if reg in Produce_reg_BB0 and reg not in Produce_reg_BB1:
                        deps = [i for i, x in enumerate(Produce_reg_BB0) if x == reg][-1] 
                        self.Dependency_tab['LoopInvar_dep'][i][reg] = Produce_ID_BB0[deps]
                        
        for i in range(BB2_ind[0], BB2_ind[1]):
            Consume_reg = self.Dependency_tab['Consume_reg'][i]
            if bool(Consume_reg):
                for reg in Consume_reg:
                    if reg in Produce_reg_BB0 and reg not in Produce_reg_BB1 and reg not in Produce_reg_BB2:
                        deps = [i for i, x in enumerate(Produce_reg_BB0) if x == reg][-1] 
                        self.Dependency_tab['LoopInvar_dep'][i][reg] = Produce_ID_BB0[deps]        
        
    # find Post Loop Dependencies
    def find_PostLoop_dep(self):
        BB1_ind = (self.loop_region[0], self.loop_region[1]+1)
        BB2_ind = (self.loop_region[1]+1, self.Dependency_tab['Instr_Addr'][-1]+1)
        Produce_reg_BB1 = self.Dependency_tab['Des_reg'][BB1_ind[0]:BB1_ind[1]]
        Produce_ID_BB1 = self.Dependency_tab['ID'][BB1_ind[0]:BB1_ind[1]]
        Produce_Instr_Addr = self.Dependency_tab['Instr_Addr'][BB1_ind[0]:BB1_ind[1]]
        
        Produce_reg_BB2 = self.Dependency_tab['Des_reg'][BB2_ind[0]:BB2_ind[1]]
        
        for i in range(BB2_ind[0], BB2_ind[1]):
            Consume_reg = self.Dependency_tab['Consume_reg'][i]
            if bool(Consume_reg):
                for reg in Consume_reg:
                    if reg in Produce_reg_BB1 and reg not in Produce_reg_BB2:
                        deps = [i for i, x in enumerate(Produce_reg_BB1) if x == reg][-1]
                        self.Dependency_tab['PostLoop_dep'][i][reg] = Produce_ID_BB1[deps]
                        
    
    def dependency_analysis(self):
        All_ins = [self.BB0, self.BB1, self.BB2]
        while len(All_ins) != 0:
            BB = All_ins.pop(0)
            for ins in BB:
                self.Dependency_tab['Instr_Addr'].append(ins['PC'])
                self.Dependency_tab['ID'].append(self.ID)
                self.schedule_addr_tab[self.ID] = None
                self.Dependency_tab['opcode'].append(ins['Ins'][0])
                self.ID += 1
                self.Dependency_tab['Des_reg'].append(self.find_Des_reg(ins['Ins']))  
                self.Dependency_tab['Consume_reg'].append(self.find_Consume_reg(ins['Ins']))
                
        self.Dependency_tab['Local_dep'] = [{} for _ in range(len(self.Dependency_tab['ID']))]
        self.find_Local_dep()
        
        self.Dependency_tab['PostLoop_dep'] = [{} for _ in range(len(self.Dependency_tab['ID']))]
        self.find_PostLoop_dep()
        
        self.Dependency_tab['LoopInvar_dep'] = [{} for _ in range(len(self.Dependency_tab['ID']))]
        self.find_LoopInvar_dep()
 
        self.Dependency_tab['InterLoop_dep'] = [{} for _ in range(len(self.Dependency_tab['ID']))]
        self.find_InterLoop_dep()
        
    def show_dependency_tab(self):
        for i in range(len(self.Dependency_tab['ID'])):
            print(self.Dependency_tab['Instr_Addr'][i], self.Dependency_tab['ID'][i],\
                  self.Dependency_tab['opcode'][i], self.Dependency_tab['Des_reg'][i], \
                  self.Dependency_tab['Local_dep'][i], self.Dependency_tab['InterLoop_dep'][i],\
                  self.Dependency_tab['LoopInvar_dep'][i], self.Dependency_tab['PostLoop_dep'][i], self.Dependency_tab['Consume_reg'][i])

    
    def check_empty_slot(self, opcode, bundle: schedule_cell):
        find_slot = False 
        slot_type = None
        if opcode in ['add', 'addi', 'sub', 'mov']:
            if bundle.ALU0_empty():
                slot_type = 'ALU0'
                find_slot = True
            elif bundle.ALU1_empty():
                slot_type = 'ALU1'
                find_slot = True
        elif opcode == 'mulu':
            if bundle.Mult_empty():
                slot_type = 'Mult'
                find_slot = True
        elif opcode in ['ld', 'st']:
            if bundle.Mem_empty():
                slot_type = 'Mem'
                find_slot = True
        else:
            if bundle.Branch_empty():
                slot_type = 'Branch'
                find_slot = True
        return find_slot, slot_type
    
    def latency(self, dep_opcode):
        lat = 1
        if dep_opcode == 'mulu':
            lat = 3
        return lat
    
    def check_dependency(self, i, dep_type: str, phase=0):
        dep_result = False
        if dep_type == 'Local_dep':
            dep_result = (len(self.Dependency_tab['Local_dep'][i]) > 0)
            
        if dep_type == 'LoopInvar_dep':
            dep_result = (len(self.Dependency_tab['LoopInvar_dep'][i]) > 0)
            
        if dep_type == 'PostLoop_dep':
            dep_result = (len(self.Dependency_tab['PostLoop_dep'][i]) > 0)
            
        # To determine interloop dep
        if dep_type == 'InterLoop_dep' and phase > 0:
            if phase == 1:
                dep_key = 'BB0'
            elif phase == 2:
                dep_key = 'BB1'
            if len(self.Dependency_tab['InterLoop_dep'][i]) > 0:
                dep = self.Dependency_tab['InterLoop_dep'][i]
                for dep_reg, dep_ID in dep.items():
                    if dep_key in dep_ID:
                        dep_result = True
                        break
        if dep_type == 'InterLoop_dep' and phase == 0:
            dep_result = (len(self.Dependency_tab['InterLoop_dep'][i]) > 0)
                        
        return dep_result
    
    def get_max_desire_addr(self, i, dep_type: str):
        max_desire_addr = 0
        # with dependency
        dep = self.Dependency_tab[dep_type][i]
        # for multiple conflicts, find the max distance
        for dep_reg, dep_ID in dep.items():
            # find producer's opcode, addr in schedule, calculate distance between producer and consumer
            if dep_type == 'InterLoop_dep':
                if 'BB0' in dep_ID:
                    dep_ID = dep_ID['BB0']
                else:
                    continue
            
            dep_opcode = self.Dependency_tab['opcode'][dep_ID]
            dep_addr = self.schedule_addr_tab[dep_ID]['addr']
            desire_addr = dep_addr + self.latency(dep_opcode)
            if desire_addr > max_desire_addr:
                max_desire_addr = desire_addr
        return max_desire_addr
    
    
    def add_empty_bubble(self, num_bubble, start_bubble_addr, insert_BB0=False):
        
        # case: reduce unnecessary delay slots within the loop body
        if insert_BB0:
            self.BB0_end = self.BB0_end + num_bubble
            self.BB1_start = self.BB1_start + num_bubble
            
        # add bubbles
        for bubble_addr in range(start_bubble_addr, start_bubble_addr+num_bubble):
            self.schedule_tab.insert(bubble_addr, schedule_cell(bubble_addr))
        
        # update addr of bundle in schedule_tab
        for new_addr in range(start_bubble_addr+num_bubble, len(self.schedule_tab)):
            self.schedule_tab[new_addr].set_addr(new_addr)
            
        # update schedule_addr_tab
        for bundle in self.schedule_tab:
            if not bundle.ALU0_empty():
                self.schedule_addr_tab[bundle.ALU0]['addr'] = bundle.addr
            if not bundle.ALU1_empty():
                self.schedule_addr_tab[bundle.ALU1]['addr'] = bundle.addr
            if not bundle.Mult_empty():
                self.schedule_addr_tab[bundle.Mult]['addr'] = bundle.addr
            if not bundle.Mem_empty():
                self.schedule_addr_tab[bundle.Mem]['addr'] = bundle.addr
            if not bundle.Branch_empty():
                self.schedule_addr_tab[bundle.Branch]['addr'] = bundle.addr
                
        return len(self.schedule_tab)-1
                    
    def find_schedule_slot(self, opcode, ID, addr, start_bundle_addr=0, end_bundle_addr=0, find_BB1_end=False):
        current_addr = addr
        find_slot = False
        for bundle_addr in range(start_bundle_addr, end_bundle_addr):
            find_slot, slot_type = self.check_empty_slot(opcode, self.schedule_tab[bundle_addr])
            if find_slot == True:
                # find empty slot, record and break
                self.schedule_tab[bundle_addr].set_slot(slot_type, ID)
                self.schedule_addr_tab[ID] = {'addr':bundle_addr, 'slot_type':slot_type}
                break
                
        # no empty slots, creat a new bundle, then record it      
        if find_slot == False:
            current_addr += 1
            if find_BB1_end:
                self.BB1_end += 1
                self.BB2_start += 1
            bundle = schedule_cell(current_addr)
            _, slot_type = self.check_empty_slot(opcode, bundle)
            bundle.set_slot(slot_type, ID)
            self.schedule_tab.append(bundle)
            self.schedule_addr_tab[ID] = {'addr':current_addr, 'slot_type':slot_type}
            
        return current_addr
        
         
    def show_scheduling_tab(self):
        for cell in self.schedule_tab:
            cell.show_schedule_cell()  
    
    def scheduling_analysis(self):

        BB0_ind = (0, self.loop_region[0])
        BB1_ind = (self.loop_region[0], self.loop_region[1]+1)
        BB2_ind = (self.loop_region[1]+1, self.Dependency_tab['Instr_Addr'][-1]+1)
        addr = 0
        bundle = schedule_cell(addr)
        self.schedule_tab.append(bundle)
        
        # first schedule instructins in BB0
        for i in range(BB0_ind[0], BB0_ind[1]):
            opcode = self.Dependency_tab['opcode'][i]
            ID = self.Dependency_tab['ID'][i]
            # check Local dependency
            if self.check_dependency(i, 'Local_dep'):
                # with dependency
                max_desire_addr = self.get_max_desire_addr(i, 'Local_dep')
                if max_desire_addr > addr:
                    # add empty bubble bundles, then record it
                    num_bubble = max_desire_addr - addr
                    addr = self.add_empty_bubble(num_bubble, addr+1)
                    addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.schedule_tab[-1].addr, end_bundle_addr=len(self.schedule_tab))
                elif max_desire_addr == addr:
                    # find empty slot in current bundle
                    addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.schedule_tab[-1].addr, end_bundle_addr=len(self.schedule_tab))
                else:
                    # max_desire_addr < addr
                    addr = self.find_schedule_slot(opcode, ID, addr, end_bundle_addr=len(self.schedule_tab))
            else:
                # without dependency, then go through the table, to find a empty slot
                addr = self.find_schedule_slot(opcode, ID, addr, end_bundle_addr=len(self.schedule_tab))
                
        self.BB0_end = addr  # record final addr of BB0
        
        # Then schedule instructins in BB1
        # a new portion for BB1
        addr += 1
        self.BB1_start = addr
        self.schedule_tab.append(schedule_cell(addr))
        for i in range(BB1_ind[0], BB1_ind[1]):
            opcode = self.Dependency_tab['opcode'][i]
            ID = self.Dependency_tab['ID'][i]
            
            if not (self.check_dependency(i, 'Local_dep') or self.check_dependency(i, 'LoopInvar_dep') or self.check_dependency(i, 'InterLoop_dep', phase=1)):
                # without dependency
                if opcode == 'loop':
                    # (1) loop instruction
                    addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.schedule_tab[-1].addr, end_bundle_addr=len(self.schedule_tab))
                else:
                    # (2) normal instruction without phase 1 dependencies 
                    addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.BB1_start, end_bundle_addr=len(self.schedule_tab))
            else:
                
                # first compare 1st round InterLoop dep and LoopInvar dep
                loop_desire_addr = None
                if self.check_dependency(i, 'LoopInvar_dep'):
                    loop_desire_addr = self.get_max_desire_addr(i, 'LoopInvar_dep')
                if self.check_dependency(i, 'InterLoop_dep', phase=1):
                    if not loop_desire_addr:
                        loop_desire_addr = self.get_max_desire_addr(i, 'InterLoop_dep')
                    else:
                        loop_desire_addr = max(loop_desire_addr, self.get_max_desire_addr(i, 'InterLoop_dep'))
                        
                # then check local dependency 
                if self.check_dependency(i, 'Local_dep'):
                    max_desire_addr = self.get_max_desire_addr(i, 'Local_dep')
                    if max_desire_addr > addr:
                        # add empty bubble bundles, then record it
                        num_bubble = max_desire_addr - addr
                        addr = self.add_empty_bubble(num_bubble, addr+1)
                        start_addr = addr
                    elif max_desire_addr == addr:
                        # find empty slot in current bundle
                        start_addr = addr
                    else:
                        # max_desire_addr < addr
                        start_addr = max_desire_addr
                        
                    # integrate all deps
                    if loop_desire_addr:
                        if loop_desire_addr <= start_addr:
                            addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=start_addr, end_bundle_addr=len(self.schedule_tab))
                        else:
                            # loop_desire_addr > start_addr
                            # add bubbles in BB0
                            num_bubble = loop_desire_addr - start_addr
                            addr = self.add_empty_bubble(num_bubble, self.BB1_start, insert_BB0=True)
                            start_addr = start_addr + num_bubble
                            addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=start_addr, end_bundle_addr=len(self.schedule_tab))
                    else:
                        # only Local dep
                        addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=start_addr, end_bundle_addr=len(self.schedule_tab))
                else:
                    # without local dependency 
                    if loop_desire_addr <= self.BB1_start:
                        addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.BB1_start, end_bundle_addr=len(self.schedule_tab))
                    else:
                        # loop_desire_addr > self.BB1_start:
                        if loop_desire_addr <= addr:
                            # loop_desire_addr <= addr:
                            addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=loop_desire_addr, end_bundle_addr=len(self.schedule_tab))
                        else:
                            # loop_desire_addr > addr:
                            num_bubble = loop_desire_addr - addr
                            addr = self.add_empty_bubble(num_bubble, self.BB1_start, insert_BB0=True)
                            addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.schedule_tab[-1].addr, end_bundle_addr=len(self.schedule_tab))
        
        self.BB1_end = addr  # record final addr of BB1
                            
        # Then schedule instructins in BB2
        # a new portion for BB2
        addr += 1
        self.BB2_start = addr
        self.schedule_tab.append(schedule_cell(addr))
        for i in range(BB2_ind[0], BB2_ind[1]):
            opcode = self.Dependency_tab['opcode'][i]
            ID = self.Dependency_tab['ID'][i]
            
            if not (self.check_dependency(i, 'Local_dep') or self.check_dependency(i, 'LoopInvar_dep') or self.check_dependency(i, 'PostLoop_dep')):
                # without dependency
                addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.BB2_start, end_bundle_addr=len(self.schedule_tab))
            else:
    
                # first compare PostLoop dep and LoopInvar dep
                loop_desire_addr = None
                if self.check_dependency(i, 'LoopInvar_dep'):
                    loop_desire_addr = self.get_max_desire_addr(i, 'LoopInvar_dep')
                if self.check_dependency(i, 'PostLoop_dep'):
                    if not loop_desire_addr:
                        loop_desire_addr = self.get_max_desire_addr(i, 'PostLoop_dep')
                    else:
                        loop_desire_addr = max(loop_desire_addr, self.get_max_desire_addr(i, 'PostLoop_dep'))
                        
                # then check local dependency 
                if self.check_dependency(i, 'Local_dep'):
                    max_desire_addr = self.get_max_desire_addr(i, 'Local_dep')
                    if max_desire_addr > addr:
                        # add empty bubble bundles, then record it
                        num_bubble = max_desire_addr - addr
                        addr = self.add_empty_bubble(num_bubble, addr+1)
                        start_addr = addr
                    elif max_desire_addr == addr:
                        start_addr = addr
                    else:
                        # max_desire_addr < addr
                        start_addr = max_desire_addr
                        
                    # integrate all deps
                    if loop_desire_addr:
                        if loop_desire_addr <= start_addr:
                            addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=start_addr, end_bundle_addr=len(self.schedule_tab))
                        else:
                            # loop_desire_addr > start_addr
                            # add bubbles in BB2
                            num_bubble = loop_desire_addr - start_addr
                            addr = self.add_empty_bubble(num_bubble, self.BB2_start)
                            start_addr = start_addr + num_bubble
                            addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=start_addr, end_bundle_addr=len(self.schedule_tab))
                    else:
                        # only Local dep
                        addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=start_addr, end_bundle_addr=len(self.schedule_tab))
                else:
                    # without local dependency 
                    if loop_desire_addr <= self.BB2_start:
                        addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.BB2_start, end_bundle_addr=len(self.schedule_tab))
                    else:
                        # loop_desire_addr > self.BB1_start:
                        if loop_desire_addr <= addr:
                            # loop_desire_addr <= addr:
                            addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=loop_desire_addr, end_bundle_addr=len(self.schedule_tab))
                        else:
                            # loop_desire_addr > addr:
                            num_bubble = loop_desire_addr - addr
                            addr = self.add_empty_bubble(num_bubble, self.BB2_start)
                            addr = self.find_schedule_slot(opcode, ID, addr, start_bundle_addr=self.schedule_tab[-1].addr, end_bundle_addr=len(self.schedule_tab))
                            
        # 2nd round: evaluate equation s(P) + lat(P) <= s(C) + II for all interLoop deps
        current_II = self.BB1_end - self.BB1_start + 1
        desire_II = []
        for i in range(BB1_ind[0], BB1_ind[1]):
            opcode = self.Dependency_tab['opcode'][i]
            ID = self.Dependency_tab['ID'][i]
            
            if self.check_dependency(i, 'InterLoop_dep', phase=2):
                # with dependency
                dep = self.Dependency_tab['InterLoop_dep'][i]
                for dep_reg, dep_ID in dep.items():
                    if 'BB1' in dep_ID:
                        producer_addr = self.schedule_addr_tab[dep_ID['BB1']]['addr']
                        producer_latency = self.latency(self.Dependency_tab['opcode'][dep_ID['BB1']])
                        consumer_addr = self.schedule_addr_tab[i]['addr']
                        desire_II.append(producer_addr + producer_latency - consumer_addr)
        if len(desire_II) > 0 and (max(desire_II) > current_II):
            loop_current_addr = self.BB1_end
            self.schedule_tab[loop_current_addr].Mem = 'nop'
            num_bubble = max(desire_II) - current_II
            addr = self.add_empty_bubble(num_bubble, self.BB2_start)
            self.BB1_end = self.BB1_end + num_bubble
            self.BB2_start = self.BB2_start + num_bubble
            # self.loop_region[1] is loop'sID
            addr = self.find_schedule_slot('loop', self.loop_region[1], addr, start_bundle_addr=self.BB1_end, end_bundle_addr=self.BB2_start, find_BB1_end = True)
            
    def check_imm(self, opcode):
        with_imm = False
        imm_ind = None
        if opcode in ['ld', 'st']:
            with_imm = True
            imm_ind = 2
        if opcode == 'addi':
            imm_ind = 3
            with_imm = True
        return with_imm, imm_ind   
    
    def find_clean_dep_slot(self, opcode, clean_ID, clean_dep, start_bundle_addr, end_bundle_addr):
        find_slot = False
        for bundle_addr in range(start_bundle_addr, end_bundle_addr):
            find_slot, slot_type = self.check_empty_slot(opcode, self.schedule_tab[bundle_addr])
            if find_slot == True:
                # find empty slot, record and break
                self.schedule_tab[bundle_addr].insert_clean_dep_slot(slot_type, clean_dep)
                self.schedule_addr_tab[clean_ID] = {'addr':bundle_addr, 'slot_type':slot_type+' clean_dep'}
                break
                
        # no empty slots, creat a new bundle, then record it      
        if find_slot == False:
            self.BB1_end += 1
            self.BB2_start += 1
            bundle = schedule_cell(end_bundle_addr)    
            _, slot_type = self.check_empty_slot(opcode, bundle)
            bundle.insert_clean_dep_slot(slot_type, clean_dep)
            self.schedule_tab.append(bundle)
            self.schedule_addr_tab[clean_ID] = {'addr':self.BB1_end, 'slot_type':slot_type+' clean_dep'}
            
            # update addr of bundle in schedule_tab
            for new_addr in range(self.BB2_start, len(self.schedule_tab)):
                self.schedule_tab[new_addr].set_addr(new_addr)
            
    def register_allocating(self):
        # phase 1: allocating registers to dest
        reg_ID = 1
        for i in range(len(self.schedule_tab)):
            for slot_type in ['ALU0', 'ALU1', 'Mult', 'Mem', 'Branch']:
                if self.schedule_tab[i].return_ID(slot_type) != 'nop':
                    ID = self.schedule_tab[i].return_ID(slot_type)
                    opcode = self.Dependency_tab['opcode'][ID]
                    Des_reg = self.Dependency_tab['Des_reg'][ID]
                    if Des_reg not in ['LC', 'EC', None]:
                        self.register_rename_tab[ID] = 'x'+str(reg_ID)
                        if bool(self.Dependency_tab['Consume_reg'][ID]):
                            # with dep
                            self.schedule_tab[i].set_slot(slot_type, {ID: [opcode, 'x'+str(reg_ID)]})
                        else:
                            # without dep
                            Ins = self.Ins_set[ID]['Ins']
                            Ins[1] = 'x'+str(reg_ID)
                            self.schedule_tab[i].set_slot(slot_type, {ID: Ins})
                        reg_ID += 1
                    elif Des_reg in ['LC', 'EC']:
                        self.schedule_tab[i].set_slot(slot_type, {ID: self.Ins_set[ID]['Ins']})
                    else:
                        # opcode = loop, st, (nop)
                        if opcode == 'loop':
                            loop_Ins = self.Ins_set[ID]['Ins']
                            loop_Ins[1] = str(self.BB1_start)
                            self.schedule_tab[i].set_slot(slot_type, {ID: loop_Ins})
                        if opcode == 'st':
                            self.schedule_tab[i].set_slot(slot_type, {ID: [opcode]})
        
        # phase 2: link consumer to producer
        for i in range(len(self.Dependency_tab['ID'])):
            if self.Dependency_tab['Consume_reg'][i]:
                link_reg = {}
                for dep_type in ['Local_dep', 'InterLoop_dep', 'LoopInvar_dep', 'PostLoop_dep']:
                    if self.check_dependency(i, dep_type):
                        # with dep
                        dep = self.Dependency_tab[dep_type][i]
                        for dep_reg, dep_ID in dep.items():
                            if dep_type == 'InterLoop_dep':
                                if 'BB0' in dep_ID:
                                    dep_ID = dep_ID['BB0']
                                else:
                                    dep_ID = dep_ID['BB1']
                                link_reg[dep_reg] = self.register_rename_tab[dep_ID]
                            else:
                                # for other deps
                                link_reg[dep_reg] = self.register_rename_tab[dep_ID]
                

                addr = self.schedule_addr_tab[i]['addr']
                slot_type = self.schedule_addr_tab[i]['slot_type']
                for consume_reg in self.Dependency_tab['Consume_reg'][i]:
                    self.schedule_tab[addr].append_consume_reg_slot(i, slot_type, link_reg[consume_reg])
                    
                opcode = self.Dependency_tab['opcode'][i]
                with_imm, imm_ind = self.check_imm(opcode)
                
                if with_imm:
                    imm = self.Ins_set[self.Dependency_tab['Instr_Addr'][i]]['Ins'][imm_ind].split('(')[0]
                    self.schedule_tab[addr].insert_imm(i, slot_type, imm_ind, imm)
                    
        
        # phase 3: fix Interloop dep
        # check InterLoop dep
        clean_InterLoop_dep = []
        for i in self.Dependency_tab['InterLoop_dep']:
            if i not in clean_InterLoop_dep:
                clean_InterLoop_dep.append(i)
        # produce instructions required in phase 3
        # find required bundle start positions
        for i in range(len(clean_InterLoop_dep)):
            dep = clean_InterLoop_dep[i]
            for dep_reg, dep_ID in dep.items():
                if 'BB0' in dep_ID and 'BB1' in dep_ID:
                    desire_start_addr = self.schedule_addr_tab[dep_ID['BB1']]['addr'] + self.latency(self.Dependency_tab['opcode'][dep_ID['BB1']])
                    self.phase_3_Ins.append({'addr':desire_start_addr, 'Ins':['mov', self.register_rename_tab[dep_ID['BB0']], self.register_rename_tab[dep_ID['BB1']]]})
        
        # insert clean_InterLoop dep into register Allocation tab
        old_BB1_end = self.BB1_end
        loop_info = self.schedule_tab[old_BB1_end].Branch
        self.schedule_tab[old_BB1_end].Branch = 'nop'
        clean_ID = -1
        for clean_dep in self.phase_3_Ins:
            if clean_dep['addr'] <= old_BB1_end:
                self.find_clean_dep_slot('mov', clean_ID, {clean_ID:clean_dep['Ins']}, start_bundle_addr=old_BB1_end, end_bundle_addr=self.BB2_start)
                clean_ID = clean_ID - 1
            else:
                if clean_dep['addr'] <= self.BB1_end:
                    self.find_clean_dep_slot('mov', clean_ID, {clean_ID:clean_dep['Ins']}, start_bundle_addr=clean_dep['addr'], end_bundle_addr=self.BB2_start)
                    clean_ID = clean_ID - 1
                else:
                    # clean_dep['addr'] > self.BB1_end
                    num_bubble = clean_dep['addr'] - self.BB1_end
                    # add bubbles
                    for bubble_addr in range(self.BB2_start, self.BB2_start+num_bubble):
                        self.schedule_tab.insert(bubble_addr, schedule_cell(bubble_addr))  
                                # update addr of bundle in schedule_tab
                    for new_addr in range(self.BB2_start, len(self.schedule_tab)):
                        self.schedule_tab[new_addr].set_addr(new_addr)
                        
                    self.BB1_end = self.BB1_end + num_bubble
                    self.BB2_start = self.BB2_start + num_bubble
                    self.find_clean_dep_slot('mov', clean_ID, {clean_ID:clean_dep['Ins']}, start_bundle_addr=self.BB1_end, end_bundle_addr=self.BB2_start)
                    clean_ID = clean_ID - 1
                    
        # change loop position
        self.schedule_tab[self.BB1_end].Branch = loop_info
        
    def output_results(self):
        for bundle in self.schedule_tab:
            single_bundle = []
            for slot_type in ['ALU0', 'ALU1', 'Mult', 'Mem', 'Branch']:
                single_bundle.append(bundle.output_schedule_cell(slot_type))
            
            self.output_bundles.append(single_bundle)
        
    def readInstructions(self, all_instructions):
        PC = 0
        while len(all_instructions) != 0:
            instruction = all_instructions.pop(0).replace(',', '').split(' ')
            DIR_cell = {'PC':PC, 'Ins':instruction}
            if instruction[0] == 'loop':
                self.loop_region = (int(instruction[1]), PC)
            self.Ins_set.append(DIR_cell)
            PC += 1

In [4]:
def give_details(compiler: Compiler_state()):
    print('Instructions: ')
    compiler.show_block()
    print('--------------')
    print('Dependency_tab: ')
    compiler.show_dependency_tab()
    print('--------------')
    print('Scheduling_tab: ')
    compiler.show_scheduling_tab()
    print('--------------')
    print('Scheduling_addr_tab: ')
    print(compiler.schedule_addr_tab)
    print('--------------')
    print('Register_allocation_tab: ')
    compiler.show_scheduling_tab()
    print('--------------')
    print('Register_rename_tab: ')
    print(compiler.register_rename_tab)
    print('--------------')
    print('Phase_3_Ins: ')
    print(compiler.phase_3_Ins)
    print('--------------')
    

In [5]:
def parseInstructions(file_name='handout.json'):
    json_file = open(file_name)
    all_instructions = json.load(json_file)    
    return all_instructions

In [6]:
def saveOutput(file_name='loop_result.json'):
    with open(file_name, 'w') as fp:
        fp.write(
            '[\n\t' + ',\n\t'.join(json.dumps(bundle) for bundle in compiler.output_bundles) + '\n]')

#### Run the Compiler for loop

In [11]:
compiler = Compiler_state()
all_instructions = parseInstructions(file_name='handout.json')
compiler.readInstructions(all_instructions)

compiler.generate_block()
compiler.dependency_analysis()
compiler.scheduling_analysis()
compiler.register_allocating()
compiler.output_results()
saveOutput(file_name='loop_result.json')